In [3]:
from scipy.spatial.transform import Rotation 
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [4]:
def Pi(p:np.ndarray) -> np.ndarray:
    """convert from homogeneous coordinates to inhomogeneous coordinates

    subtract one coordinate"""
    if isinstance(p, np.ndarray):
        return p[:-1]/p[-1]
    elif isinstance(p, list):
        return [Pi(np.array(p_)) for p_ in p]

def PiInv(p:np.ndarray) -> np.ndarray:
    """convert from inhomogeneous coordinates to homogeneous coordinates

    add one coordinate with value 1"""
    if isinstance(p, np.ndarray):
        return np.vstack((p, np.ones(p.shape[1])))
    elif isinstance(p, list):
        return [PiInv(np.array(p_)) for p_ in p]

def projectpoints(K, R, t, Q):
    """project 3D points to 2D"""
    Rt = np.concatenate((R, t), axis=1)
    return Pi(K @ Rt @ PiInv(Q))

In [5]:
R2 = Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix()
K = np.array([[1000, 0, 300], [0, 1000, 200], [0, 0, 1]])
R1 = np.eye(3)
t1 = np.array([[0, 0, 0]]).T
t2 = np.array([[0.2, 2, 1]]).T
Cam1 = np.concatenate((R1, t1), axis=1)
Cam2 = np.concatenate((R2, t2), axis=1)

In [6]:
# Ex 3.1
Q = np.array([[1,0.5,4,1]]).T 
# 3D point in homogeneous coordinates a bit silly since I'm converting back to imhomogeneous coordinates to use with projectpoints
q1 = projectpoints(K, R1, t1, Pi(Q))
q2 = projectpoints(K, R2, t2, Pi(Q))

In [7]:
q1, q2 # correct

(array([[550.],
        [325.]]),
 array([[582.47256835],
        [185.98985776]]))

In [8]:
# 3.2
def crossOp(a:np.ndarray) -> np.ndarray:
    """crossproduct operator"""
    x = a[0].item()
    y = a[1].item()
    z = a[2].item()
    return np.array([[0, -z, y], [z, 0, -x], [-y, x, 0]])
# check that it works by comparing with numpy.cross
x = np.array([[3,5,8]]).T
y = np.array([[9,8,2]]).T
crossOp(x) @ y, np.cross(x.ravel(),y.ravel())

(array([[-54],
        [ 66],
        [-21]]),
 array([-54,  66, -21]))

In [9]:
# 3.3
def compute_fundamental_matrix(K1, K2, R2, t2):
    """compute fundamental matrix from camera matrices"""
    E = crossOp(t2) @ R2
    F = np.linalg.inv(K2).T @ E @ np.linalg.inv(K1)
    return F
F = compute_fundamental_matrix(K, K, R2, t2)
F

array([[ 3.29311881e-07,  8.19396327e-07,  1.79162592e-03],
       [ 5.15532551e-07, -8.76915984e-07,  9.31426656e-05],
       [-1.29882755e-03,  1.51951700e-03, -1.10072682e+00]])

In [10]:
PiInv(q2).T.ravel() @ F @ PiInv(q1).ravel() # should be zero

4.440892098500626e-16

In [11]:
# 3.4 epipolar lines
def compute_epipolar_lines(F, Q):
    """compute epipolar lines for each point in Q"""
    return F @ PiInv(Q)
l1 = compute_epipolar_lines(F, q1)
l1

array([[ 2.23905126e-03],
       [ 9.16878739e-05],
       [-1.32123895e+00]])

In [12]:
Pi(l1), Pi(np.array([[8.956e-3, 3.668e-4, -5.285]]).T) # check with solution is it correct

(array([[-1.69466035e-03],
        [-6.93953760e-05]]),
 array([[-1.69460738e-03],
        [-6.94039735e-05]]))

In [13]:
# 3.5 q2 on the epipolar lines of 3.4
PiInv(q2).T @ l1 # should be 0
# why this is the case: both the point q2 and the line l are derived from the same 3D point Q. This 3D point yields a single epipolar plane, and the plane yields a single line in each camera. The projections of the 3D point must lie on the epipolar lines.

array([[4.4408921e-16]])

In [59]:
# 3.6 analytically
# 3.7 analytically
# 3.8
data = np.load('data/TwoImageDataCar.npy', allow_pickle=True).item()
im1, im2, R1, R2, t1, t2, K = data['im1'], data['im2'], data['R1'], data['R2'], data['t1'], data['t2'], data['K']

In [60]:
F = compute_fundamental_matrix(K, K, R2, t2)

In [61]:
# 3.9
def DrawLine(l, shape, ax):
    """Checks where the line intersects the four sides of the image and finds the two intersections that are within the frame"""
    def in_frame(l_im):
        q = np.cross(l.flatten(), l_im)
        q = q[:2]/q[2]
        if all(q>=0) and all(q+1<=shape[1::-1]):
            return q
    lines = [[1, 0, 0], [0, 1, 0], [1, 0, 1-shape[1]], [0, 1, 1-shape[0]]]
    P = [in_frame(l_im) for l_im in lines if in_frame(l_im) is not None]
    ax.plot(*np.array(P).T, 'r-')

In [63]:
%matplotlib qt
fig, ax = plt.subplots(1,2, figsize=(10,5))
fig.suptitle('Click on a point in im1 to draw the epipolar line in im2')
ax[0].imshow(im1, cmap='gray'); ax[0].set_title('im1')
ax[1].imshow(im2, cmap='gray'); ax[1].set_title('im2')
x, y = plt.ginput(1, mouse_stop='q',timeout=5)[0]

ax[0].plot(x, y, 'r+')
l = F @ PiInv(np.array([[x, y]]).T)
DrawLine(l, im2.shape, ax[1])

In [67]:
# 3.10
fig, ax = plt.subplots(1,2, figsize=(10,5))
fig.suptitle('Click on a point in im2 to draw the epipolar line in im1')
ax[0].imshow(im1, cmap='gray'); ax[0].set_title('im1')
ax[1].imshow(im2, cmap='gray'); ax[1].set_title('im2')
x, y = plt.ginput(1, mouse_stop='q',timeout=5)[0]

ax[1].plot(x, y, 'r+')
# I'm not sure if you actually need to do this
# since there seems to be no visual difference
# e.q. (19) from exercises
R2t = R2 @ R1.T 
t2t = t2 - R2t @ t1
F = compute_fundamental_matrix(K, K, R2t, t2t)
l = F @ PiInv(np.array([[x, y]]).T)
DrawLine(l, im1.shape, ax[0])

In [217]:
# 3.11
%matplotlib inline
def triangulate(qs: list, Ps:list):
    """triangulate points from multiple cameras"""
    B = np.zeros((len(Ps)*2, Ps[0].shape[1]))
    for i, (q, p) in enumerate(zip(qs, Ps)):
        x, y = q
        x, y = x.item(), y.item()
        B[i*2, :] = p[2]*x - p[0]
        B[(i*2)+1, :] = p[2]*y - p[1]
    u, s, vh = np.linalg.svd(B)
    v = vh.T
    Q = v[:,-1]
    return Q.T / Q[-1] # not sure about dividing since the norm is no longer 1.

In [220]:
q1 = np.array([[300, 160]]).T
q2 = np.array([[300, 640]]).T
P1 = np.array([[800, 0, 300, 0], 
               [0, 800, 400, -2400],
               [0, 0, 1, 0]])
P2 = np.array([[800, 0, 300, 0],
               [0, 800, 400, 2400],
               [0, 0, 1, 0]])

Q = triangulate([q1, q2], [P1, P2])
Q.shape
Q.reshape(4,1) # solution = [0, 0, 10, 1]

array([[ 0.],
       [ 0.],
       [10.],
       [ 1.]])

In [221]:
np.linalg.norm(Q)

10.04987562112089